In [1]:
# Loads S_sales in Walmart data
spark.read.option('header', True).option('inferSchema', True) \
  .csv('./dataset/RealWorldDatasets/WalMart/S_sales.csv').createOrReplaceTempView('S_sales')

S_sales = spark.sql(
    "SELECT " \
        "(CAST(TRIM(BOTH '\\'' FROM weekly_sales) AS INT) - 1) weekly_sales, " \
        "CAST(TRIM(BOTH '\\'' FROM sid) AS INT) sid, " \
        "CAST(TRIM(BOTH '\\'' FROM dept) AS INT) dept, " \
        "CAST(TRIM(BOTH '\\'' FROM store) AS INT) store, " \
        "HASH(purchaseid) purchaseid " \
    "FROM " \
        "S_sales s").toPandas()

In [2]:
S_sales.head(1)

,weekly_sales,sid,dept,store,purchaseid
0,3,159739,1,42,-518970915


In [3]:
import pandas_profiling

profile = S_sales.sample(n=100).profile_report(title='Pandas Profiling Report', style={'full_width':True})
profile.to_file(output_file='walmart-sales-pandas-profile.html')

In [11]:
# Loads R1_indicators in Walmart data
spark.read.option('header', True).option('inferSchema', True) \
  .csv('./dataset/RealWorldDatasets/WalMart/R1_indicators.csv').createOrReplaceTempView('R1_indicators')

R1_indicators = spark.sql(
    "SELECT " \
        "HASH(purchaseid) purchaseid, " \
        # "temperature_avg, " \
        "temperature_stdev, " \
        "fuel_price_avg, " \
        "fuel_price_stdev, " \
        "cpi_avg, " \
        "cpi_stdev, " \
        "unemployment_avg, " \
        "unemployment_stdev, " \
        "holidayfreq " \
    "FROM " \
        "R1_indicators").toPandas()

In [18]:
R1_indicators.head(1)

,purchaseid,temperature_stdev,fuel_price_avg,fuel_price_stdev,cpi_avg,cpi_stdev,unemployment_avg,unemployment_stdev,holidayfreq
0,74344712,41.76,0.0,3.158,0.0,218.605037,0.0,7.441,0.0


In [14]:
profile = R1_indicators.sample(n=100).profile_report(title='Pandas Profiling Report', style={'full_width':True})
profile.to_file(output_file='walmart-indicators-pandas-profile.html')

In [19]:
# Loads R2_stores in Walmart data
spark.read.option('header', True).option('inferSchema', True) \
  .csv('./dataset/RealWorldDatasets/WalMart/R2_stores.csv').createOrReplaceTempView('R2_stores')

R2_stores = spark.sql(
    "SELECT " \
        "CAST(TRIM(BOTH '\\'' FROM store) AS INT) store, " \
        "CAST(TRIM(BOTH '\\'' FROM type) AS INT) type, " \
        "size " \
    "FROM " \
        "R2_stores").toPandas()

In [20]:
R2_stores.head(1)

,store,type,size
0,1,1,151315


In [22]:
profile = R2_stores.profile_report(title='Pandas Profiling Report', style={'full_width':True})
profile.to_file(output_file='walmart-stores-pandas-profile.html')

In [25]:
import featuretools as ft

# Generates EntitySet
es = ft.EntitySet(id='walmart')

In [26]:
# Adds Entities
es.entity_from_dataframe(entity_id='S_sales', dataframe=S_sales, index='sid')
es.entity_from_dataframe(entity_id='R1_indicators', dataframe=R1_indicators, index='purchaseid')
es.entity_from_dataframe(entity_id='R2_stores', dataframe=R2_stores, index='store')

Entityset: walmart
  Entities:
    S_sales [Rows: 421570, Columns: 5]
  Relationships:
    No relationships

Entityset: walmart
  Entities:
    S_sales [Rows: 421570, Columns: 5]
    R1_indicators [Rows: 2340, Columns: 9]
  Relationships:
    No relationships

Entityset: walmart
  Entities:
    S_sales [Rows: 421570, Columns: 5]
    R1_indicators [Rows: 2340, Columns: 9]
    R2_stores [Rows: 45, Columns: 3]
  Relationships:
    No relationships

In [28]:
# Generates relationship
r_indicators = ft.Relationship(es['R1_indicators']['purchaseid'], es['S_sales']['purchaseid'])
r_stores = ft.Relationship(es['R2_stores']['store'], es['S_sales']['store'])

# Defines relationship
es.add_relationships(relationships=[r_indicators, r_stores])

Entityset: walmart
  Entities:
    S_sales [Rows: 421570, Columns: 5]
    R1_indicators [Rows: 2340, Columns: 9]
    R2_stores [Rows: 45, Columns: 3]
  Relationships:
    S_sales.purchaseid -> R1_indicators.purchaseid
    S_sales.store -> R2_stores.store

In [29]:
es['S_sales'].variables

[<Variable: sid (dtype = index)>,
 <Variable: weekly_sales (dtype = numeric)>,
 <Variable: dept (dtype = numeric)>,
 <Variable: store (dtype = id)>,
 <Variable: purchaseid (dtype = id)>]

In [30]:
es['R1_indicators'].variables

[<Variable: purchaseid (dtype = index)>,
 <Variable: temperature_stdev (dtype = numeric)>,
 <Variable: fuel_price_avg (dtype = numeric)>,
 <Variable: fuel_price_stdev (dtype = numeric)>,
 <Variable: cpi_avg (dtype = numeric)>,
 <Variable: cpi_stdev (dtype = numeric)>,
 <Variable: unemployment_avg (dtype = numeric)>,
 <Variable: unemployment_stdev (dtype = numeric)>,
 <Variable: holidayfreq (dtype = numeric)>]

In [31]:
es['R2_stores'].variables

[<Variable: store (dtype = index)>,
 <Variable: type (dtype = numeric)>,
 <Variable: size (dtype = numeric)>]

In [35]:
# Runs Deep Feature Synthesis
df_feature, features_defs = ft.dfs(
    entityset=es,
    target_entity='S_sales',
    agg_primitives=['sum', 'min', 'max', 'count'],
    max_depth=1,
    chunk_size=0.025,
    verbose=True)

df_feature.head()

Built 14 features
Elapsed: 00:04 | Progress: 100%|██████████| Remaining: 00:00


,weekly_sales,dept,store,purchaseid,R1_indicators.temperature_stdev,R1_indicators.fuel_price_avg,R1_indicators.fuel_price_stdev,R1_indicators.cpi_avg,R1_indicators.cpi_stdev,R1_indicators.unemployment_avg,R1_indicators.unemployment_stdev,R1_indicators.holidayfreq,R2_stores.type,R2_stores.size
sid,,,,,,,,,,,,,,
1,4,5,34,897768995,51.213333,8.895405,3.209333,0.403288,129.218680,2.316080,9.732333,0.774270,1,158114
2,5,23,10,-1649444077,53.190000,2.237588,3.379000,0.344004,128.282773,1.925584,8.684667,1.111189,2,126512
3,5,90,39,-2076543225,70.566667,2.608799,3.204333,0.411940,216.344688,5.799426,7.473333,1.143478,1,184109
4,6,92,13,1125084295,79.256667,1.970135,3.331667,0.459926,128.639195,2.353067,6.864333,1.093055,1,219622
5,5,97,20,-1691746670,26.766667,9.585595,3.163667,0.371727,207.275667,4.733733,7.497000,0.627340,1,203742
